Hotel Prices Webscraper Decolar x Booking

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from unidecode import unidecode
from time import sleep
import pandas as pd
import os

In [2]:
# ID destinos Decolar
  #Aracaju: 180
  #Arraial dAjuda: 339
  #Balneario Camboriu: 192594
  #Belo Horizonte: 701
  #Brasilia: 926
  #Buzios: 1077
  #Cabo Frio: 1227
  #Caldas Novas: 1364
  #Campinas: 1467
  #Campos do Jordao: 6116
  #Canela: 6064
  #Curitiba: 1595
  #Florianopolis: 2261
  #Fortaleza: 2302
  #Foz do Iguacu: 3072
  #Gramado: 2611
  #Ilhabela: 3027
  #Ilheus: 3158
  #Joao Pessoa: 3399
  #Maceio: 4430
  #Morro de Sao Paulo: 4678
  #Natal: 4971
  #Porto Alegre: 5822
  #Porto de Galinhas: 5648
  #Porto Seguro: 881
  #Praia de Pipa: 5875
  #Recife: 6322
  #Rio de Janeiro: 6381
  #Salvador: 7018
  #Sao Paulo: 6574

In [3]:
# Parâmetros

nome_destino = 'Balneario Camboriu'
id_destino_decolar = '192594'
checkin = '2021-11-28' # Formato AAAA-MM-DD
checkout = '2021-11-30' # Formato AAAA-MM-DD
pax = '2' # Número de hóspedes (adultos)

In [4]:
# Definições Chromedriver

chrome_options = Options()
chrome_options.add_experimental_option(
    'excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')

In [5]:
# Scrape Booking

nome_hoteis_bkg = []
preco_hoteis_bkg = []

class ScrappyBkg:

    def iniciar(self):
        self.raspagem_de_dados()

    def raspagem_de_dados(self):
        cin = {"year": checkin[0:4],
               "month": checkin[5:7],
               "day": checkin[8:10]}
        cout = {"year": checkout[0:4],
                "month": checkout[5:7],
                "day": checkout[8:10]}
        chd = "0"
        rooms = "1"

        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://www.booking.com/searchresults.pt-br.html?label=gen173nr-1FCAEoggI46AdIM1gEaCCIAQGYAS24ARnIAQzYAQHoAQH4AQuIAgGoAgO4At_SvIsGwAIB0gIkNzI1ZmZlYWEtZWRiYS00OWZmLWI5MzItZjYxNWMwYmY3N2U02AIG4AIB&lang=pt-br&sid=7b8aa653793caf99eb3d54cf87f15296&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.pt-br.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaCCIAQGYAS24ARnIAQzYAQHoAQH4AQuIAgGoAgO4At_SvIsGwAIB0gIkNzI1ZmZlYWEtZWRiYS00OWZmLWI5MzItZjYxNWMwYmY3N2U02AIG4AIB%3Bsid%3D7b8aa653793caf99eb3d54cf87f15296%3Bsb_price_type%3Dtotal%3Bsig%3Dv1dXShAKR_%26%3B&ss={nome_destino}&is_ski_area=0&ssne={nome_destino}&ssne_untouched={nome_destino}&dest_id=-&dest_type=city&checkin_year={cin["year"]}&checkin_month={cin["month"]}&checkin_monthday={cin["day"]}&checkout_year={cout["year"]}&checkout_month={cout["month"]}&checkout_monthday={cout["day"]}&group_adults={pax}&group_children={chd}0&no_rooms={rooms}&b_h4u_keep_filters=&from_sf=1&order=review_score_and_price'
        self.lista_nome_hoteis = []
        self.lista_preco_hoteis = []
        self.driver.get(self.link)
        #self.driver.find_element(By.XPATH, '//*[@id="ajaxsrwrap"]/div[2]/div/div/div[2]/ul/li[3]/a').click()
        sleep(3)
        
        for p in range(20): # Passar como parâmetro nº de páginas a percorrer. Nº > total, percorre todas.
            item = 1
            lista_length = self.driver.find_elements(By.CLASS_NAME, 'e75f1d9859')
            lista_max = len(lista_length)
            for i in range(lista_max):
                c = 1
                desc = 1
                while c < lista_max:
                    try:
                        lista_nomes = self.driver.find_elements(By.XPATH,
                            f'//*[@id="search_results_table"]/div[1]/div/div/div/div[5]/div[{item}]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div/h3/a/div[1]')
                        self.lista_nome_hoteis.append(lista_nomes[0].text)
                        sleep(1)
                        lista_precos = self.driver.find_elements(By.XPATH,
                            f'//*[@id="search_results_table"]/div[1]/div/div/div/div[5]/div[{item}]/div[1]/div[2]/div/div[3]/div/div[2]/div/div[1]/div/div/div/div/div[2]/span')                        
                        while len(lista_precos) == 0:
                            try:
                                lista_precos = self.driver.find_elements(By.XPATH,
                                    f'//*[@id="search_results_table"]/div[1]/div/div/div/div[5]/div[{desc}]/div[1]/div[2]/div/div[4]/div/div[2]/div/div[1]/div/div/div/div/div[2]/span[2]')
                                desc += 1
                            except:
                                desc += 1
                        self.lista_preco_hoteis.append(lista_precos[0].text)          
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1
            try:
                botao_proximo = self.driver.find_element(By.XPATH,
                    '//*[@id="search_results_table"]/div[1]/div/div/div/div[6]/div[2]/nav/div/div[3]/button')
                botao_proximo.click()
                print(f'\u001b[32m{"Navegando para a próxima página"}\u001b[0m')
                sleep(5)
            except:
                self.driver.quit()
        
        print(f'\u001b[32m{"Resultados:"}\u001b[0m')
        print(self.lista_nome_hoteis)
        print(self.lista_preco_hoteis)

        for nome in self.lista_nome_hoteis:
            nome_hoteis_bkg.append(nome)
        for preco in self.lista_preco_hoteis:
            preco_hoteis_bkg.append(preco)           

start = ScrappyBkg()
start.iniciar()

Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Resultados:
['Lindo apto, 100metros da praia, na Av Brasil com a Av Central melhor localização', 'Quarto de casal Romântico com Terraço', 'Morada do Estaleiro', 'APTO BC - 500m da praia', '3 SUÍTES LUXUOSO LINDA VISTA MAR A 50m DA PRAIA', 'Casa aconchegante com piscina', 'Silêncio e Privacidade', 'Recanto Sete Mares PARA FAMÍLIAS', 'Recanto dos Passarinhos',

In [6]:
# Lista nome hotéis

print(nome_hoteis_bkg)

['Lindo apto, 100metros da praia, na Av Brasil com a Av Central melhor localização', 'Quarto de casal Romântico com Terraço', 'Morada do Estaleiro', 'APTO BC - 500m da praia', '3 SUÍTES LUXUOSO LINDA VISTA MAR A 50m DA PRAIA', 'Casa aconchegante com piscina', 'Silêncio e Privacidade', 'Recanto Sete Mares PARA FAMÍLIAS', 'Recanto dos Passarinhos', 'Quitinete Balneário', 'Residencial Aroeira', 'Camping BC - Suítes - Trailer - Tiny house', 'Apartment Lugano Miguel Matte', 'Apto na Quadra do Mar e Vista para o Rio', 'Apto Central para Família', 'Apartamento Amplo em Balneário Camboriú', 'Pousada Bella Casa', 'Apto 2 quartos WI-Fi grátis centro perto da praia B Camboriú', 'APTO 2 QUARTOS AV CENTRAL', 'Flat Balneario Camboriu', 'Apto 2 quartos, 1 quadra do Mar/Shop. Atlantico', 'Apartamento Balneário Camboriu', 'Altos do Estaleiro- Balneário Camboriu', 'PORTAL DAS ROSAS', 'Aconchegante apartamento, com excelente vista mar', 'maria julia', 'moradia garden village', 'Rizzu Marina Hotel', 'Apto

In [7]:
# Tratamento formato

nome_hoteis_bkg_temp = list(pd.Series(nome_hoteis_bkg).str.upper())

nome_hoteis_bkg_fmtg = []

for nome in nome_hoteis_bkg_temp:
    nome_hoteis_bkg_fmtg.append(unidecode(nome))

print(nome_hoteis_bkg_fmtg)

['LINDO APTO, 100METROS DA PRAIA, NA AV BRASIL COM A AV CENTRAL MELHOR LOCALIZACAO', 'QUARTO DE CASAL ROMANTICO COM TERRACO', 'MORADA DO ESTALEIRO', 'APTO BC - 500M DA PRAIA', '3 SUITES LUXUOSO LINDA VISTA MAR A 50M DA PRAIA', 'CASA ACONCHEGANTE COM PISCINA', 'SILENCIO E PRIVACIDADE', 'RECANTO SETE MARES PARA FAMILIAS', 'RECANTO DOS PASSARINHOS', 'QUITINETE BALNEARIO', 'RESIDENCIAL AROEIRA', 'CAMPING BC - SUITES - TRAILER - TINY HOUSE', 'APARTMENT LUGANO MIGUEL MATTE', 'APTO NA QUADRA DO MAR E VISTA PARA O RIO', 'APTO CENTRAL PARA FAMILIA', 'APARTAMENTO AMPLO EM BALNEARIO CAMBORIU', 'POUSADA BELLA CASA', 'APTO 2 QUARTOS WI-FI GRATIS CENTRO PERTO DA PRAIA B CAMBORIU', 'APTO 2 QUARTOS AV CENTRAL', 'FLAT BALNEARIO CAMBORIU', 'APTO 2 QUARTOS, 1 QUADRA DO MAR/SHOP. ATLANTICO', 'APARTAMENTO BALNEARIO CAMBORIU', 'ALTOS DO ESTALEIRO- BALNEARIO CAMBORIU', 'PORTAL DAS ROSAS', 'ACONCHEGANTE APARTAMENTO, COM EXCELENTE VISTA MAR', 'MARIA JULIA', 'MORADIA GARDEN VILLAGE', 'RIZZU MARINA HOTEL', 'APTO

In [8]:
# Lista preço hotéis

print(preco_hoteis_bkg)

['R$ 396', 'R$ 500', 'R$ 609', 'R$ 710', 'R$ 1.300', 'R$ 1.400', 'R$ 6.000', 'R$ 270', 'R$ 310', 'R$ 338', 'R$ 378', 'R$ 400', 'R$ 460', 'R$ 480', 'R$ 480', 'R$ 500', 'R$ 520', 'R$ 600', 'R$ 600', 'R$ 600', 'R$ 680', 'R$ 700', 'R$ 760', 'R$ 800', 'R$ 900', 'R$ 910', 'R$ 1.000', 'R$ 1.070', 'R$ 1.071', 'R$ 1.160', 'R$ 1.200', 'R$ 1.200', 'R$ 1.208', 'R$ 1.300', 'R$ 3.300', 'R$ 252', 'R$ 338', 'R$ 348', 'R$ 360', 'R$ 390', 'R$ 400', 'R$ 400', 'R$ 400', 'R$ 450', 'R$ 496', 'R$ 500', 'R$ 500', 'R$ 500', 'R$ 500', 'R$ 520', 'R$ 528', 'R$ 540', 'R$ 598', 'R$ 600', 'R$ 680', 'R$ 700', 'R$ 700', 'R$ 740', 'R$ 760', 'R$ 780', 'R$ 798', 'R$ 800', 'R$ 809', 'R$ 812', 'R$ 860', 'R$ 900', 'R$ 972', 'R$ 1.380', 'R$ 1.530', 'R$ 1.998', 'R$ 2.290', 'R$ 258', 'R$ 268', 'R$ 288', 'R$ 288', 'R$ 298', 'R$ 340', 'R$ 500', 'R$ 352', 'R$ 358', 'R$ 378', 'R$ 380', 'R$ 393', 'R$ 398', 'R$ 398', 'R$ 400', 'R$ 408', 'R$ 440', 'R$ 448', 'R$ 450', 'R$ 454', 'R$ 472', 'R$ 495', 'R$ 510', 'R$ 520', 'R$ 520', 'R$ 524

In [9]:
# Tratamento formato

preco_hoteis_bkg_nofloat = []
for x in preco_hoteis_bkg:
    item = x
    for y in ["."]:
        item = item.replace(y, "")
    preco_hoteis_bkg_nofloat.append(item)
print(preco_hoteis_bkg_nofloat)

['R$ 396', 'R$ 500', 'R$ 609', 'R$ 710', 'R$ 1300', 'R$ 1400', 'R$ 6000', 'R$ 270', 'R$ 310', 'R$ 338', 'R$ 378', 'R$ 400', 'R$ 460', 'R$ 480', 'R$ 480', 'R$ 500', 'R$ 520', 'R$ 600', 'R$ 600', 'R$ 600', 'R$ 680', 'R$ 700', 'R$ 760', 'R$ 800', 'R$ 900', 'R$ 910', 'R$ 1000', 'R$ 1070', 'R$ 1071', 'R$ 1160', 'R$ 1200', 'R$ 1200', 'R$ 1208', 'R$ 1300', 'R$ 3300', 'R$ 252', 'R$ 338', 'R$ 348', 'R$ 360', 'R$ 390', 'R$ 400', 'R$ 400', 'R$ 400', 'R$ 450', 'R$ 496', 'R$ 500', 'R$ 500', 'R$ 500', 'R$ 500', 'R$ 520', 'R$ 528', 'R$ 540', 'R$ 598', 'R$ 600', 'R$ 680', 'R$ 700', 'R$ 700', 'R$ 740', 'R$ 760', 'R$ 780', 'R$ 798', 'R$ 800', 'R$ 809', 'R$ 812', 'R$ 860', 'R$ 900', 'R$ 972', 'R$ 1380', 'R$ 1530', 'R$ 1998', 'R$ 2290', 'R$ 258', 'R$ 268', 'R$ 288', 'R$ 288', 'R$ 298', 'R$ 340', 'R$ 500', 'R$ 352', 'R$ 358', 'R$ 378', 'R$ 380', 'R$ 393', 'R$ 398', 'R$ 398', 'R$ 400', 'R$ 408', 'R$ 440', 'R$ 448', 'R$ 450', 'R$ 454', 'R$ 472', 'R$ 495', 'R$ 510', 'R$ 520', 'R$ 520', 'R$ 524', 'R$ 720', 'R$

In [10]:
# Tratamento formato

preco_hoteis_bkg_nosign = []

for x in preco_hoteis_bkg_nofloat:
    item = x
    for y in ["R$ "]:
        item = item.replace(y, "")
    preco_hoteis_bkg_nosign.append(item)
print(preco_hoteis_bkg_nosign)

['396', '500', '609', '710', '1300', '1400', '6000', '270', '310', '338', '378', '400', '460', '480', '480', '500', '520', '600', '600', '600', '680', '700', '760', '800', '900', '910', '1000', '1070', '1071', '1160', '1200', '1200', '1208', '1300', '3300', '252', '338', '348', '360', '390', '400', '400', '400', '450', '496', '500', '500', '500', '500', '520', '528', '540', '598', '600', '680', '700', '700', '740', '760', '780', '798', '800', '809', '812', '860', '900', '972', '1380', '1530', '1998', '2290', '258', '268', '288', '288', '298', '340', '500', '352', '358', '378', '380', '393', '398', '398', '400', '408', '440', '448', '450', '454', '472', '495', '510', '520', '520', '524', '720', '555', '556', '558', '570', '585', '600', '600', '627', '630', '672', '798', '682', '700', '744', '751', '790', '960', '1000', '1000', '1000', '1300', '2290', '255', '280', '285', '286', '304', '337', '344', '348', '370', '371', '400', '418', '430', '440', '446', '454', '473', '500', '504', '535'

In [11]:
# Tratamento formato

preco_hoteis_bkg_nosign = pd.to_numeric(preco_hoteis_bkg_nosign, errors='coerce', downcast='unsigned')
print(preco_hoteis_bkg_nosign)

[ 396  500  609  710 1300 1400 6000  270  310  338  378  400  460  480
  480  500  520  600  600  600  680  700  760  800  900  910 1000 1070
 1071 1160 1200 1200 1208 1300 3300  252  338  348  360  390  400  400
  400  450  496  500  500  500  500  520  528  540  598  600  680  700
  700  740  760  780  798  800  809  812  860  900  972 1380 1530 1998
 2290  258  268  288  288  298  340  500  352  358  378  380  393  398
  398  400  408  440  448  450  454  472  495  510  520  520  524  720
  555  556  558  570  585  600  600  627  630  672  798  682  700  744
  751  790  960 1000 1000 1000 1300 2290  255  280  285  286  304  337
  344  348  370  371  400  418  430  440  446  454  473  500  504  535
  542  548  580  594  835  630  698  781  720  750  788  858  900  982
 1282  170  200  218  240  260  298  300  454  358  360  380  398  416
  420  476  559  578  658  238  240  311  398  300  360  372  517 2300
  420  266  368  528  678 3103  541 1120  198  200  216  270  333  400
  440 

In [12]:
# Mesclagem nomes e preços em dicionário

data_hoteis_bkg = []
for el in zip(nome_hoteis_bkg_fmtg, preco_hoteis_bkg_nosign):
    data_hoteis_bkg.append(el)
data_hoteis_bkg_dict = dict(data_hoteis_bkg)
print(data_hoteis_bkg_dict)

{'LINDO APTO, 100METROS DA PRAIA, NA AV BRASIL COM A AV CENTRAL MELHOR LOCALIZACAO': 396, 'QUARTO DE CASAL ROMANTICO COM TERRACO': 500, 'MORADA DO ESTALEIRO': 609, 'APTO BC - 500M DA PRAIA': 710, '3 SUITES LUXUOSO LINDA VISTA MAR A 50M DA PRAIA': 1300, 'CASA ACONCHEGANTE COM PISCINA': 1400, 'SILENCIO E PRIVACIDADE': 6000, 'RECANTO SETE MARES PARA FAMILIAS': 270, 'RECANTO DOS PASSARINHOS': 310, 'QUITINETE BALNEARIO': 338, 'RESIDENCIAL AROEIRA': 378, 'CAMPING BC - SUITES - TRAILER - TINY HOUSE': 400, 'APARTMENT LUGANO MIGUEL MATTE': 460, 'APTO NA QUADRA DO MAR E VISTA PARA O RIO': 480, 'APTO CENTRAL PARA FAMILIA': 480, 'APARTAMENTO AMPLO EM BALNEARIO CAMBORIU': 500, 'POUSADA BELLA CASA': 520, 'APTO 2 QUARTOS WI-FI GRATIS CENTRO PERTO DA PRAIA B CAMBORIU': 600, 'APTO 2 QUARTOS AV CENTRAL': 600, 'FLAT BALNEARIO CAMBORIU': 600, 'APTO 2 QUARTOS, 1 QUADRA DO MAR/SHOP. ATLANTICO': 680, 'APARTAMENTO BALNEARIO CAMBORIU': 500, 'ALTOS DO ESTALEIRO- BALNEARIO CAMBORIU': 760, 'PORTAL DAS ROSAS': 800

In [13]:
# Conversão dicionário para dataframe

df_data_hoteis_bkg = pd.DataFrame(list(data_hoteis_bkg_dict.items()),
                   columns=['Nome', 'Preço Booking'])
display(df_data_hoteis_bkg)

,Nome,Preço Booking
0,"LINDO APTO, 100METROS DA PRAIA, NA AV BRASIL C...",396
1,QUARTO DE CASAL ROMANTICO COM TERRACO,500
2,MORADA DO ESTALEIRO,609
3,APTO BC - 500M DA PRAIA,710
4,3 SUITES LUXUOSO LINDA VISTA MAR A 50M DA PRAIA,1300
...,...,...
223,LINDO APARTAMENTO FRENTE MAR!,3240
224,CASA DE PRAIA,3300
225,APTO NA PRAIA CENTRAL EM BALNEARIO CAMBORIU SC,3446
226,COBERTURA DUPLEX BALNEARIO CAMBORIU,5400


In [14]:
# Scrape Decolar

nome_hoteis_desp = []
preco_hoteis_desp = []

class ScrappyDesp:

    def iniciar(self):
        self.raspagem_de_dados()

    def raspagem_de_dados(self):
        page = 1
        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://www.decolar.com/accommodations/results/CIT_{id_destino_decolar}/{checkin}/{checkout}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}'
        self.lista_nome_hoteis = []
        self.lista_preco_hoteis = []
        self.driver.get(self.link)
        self.driver.find_element(By.XPATH, '//*[@id="lgpd-banner"]/div/a[2]').click()
        sleep(3)

        for p in range(20): # Passar como parâmetro nº de páginas a percorrer. Nº > total, percorre todas.
            item = 1
            lista_length = self.driver.find_elements(By.CLASS_NAME, 'accommodation-name')
            lista_max = len(lista_length)
            for i in range(lista_max):
                c = 1
                while c < lista_max:
                    try:
                        lista_nomes = self.driver.find_elements(By.XPATH,
                            f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[1]/div/div[2]/div/aloha-cluster-accommodation-info-container/div[1]/span')
                        self.lista_nome_hoteis.append(lista_nomes[0].text)
                        sleep(1)
                        lista_precos = self.driver.find_elements(By.XPATH,
                            f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[2]/aloha-cluster-pricebox-container/div/div[2]/div[1]/aloha-price-container/aloha-summary-price/div/span[2]')
                        self.lista_preco_hoteis.append(lista_precos[0].text)          
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1                        
            try:
                page += 1
                self.driver.get(f'https://www.decolar.com/accommodations/results/CIT_{id_destino_decolar}/{checkin}/{checkout}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}')
                print(f'\u001b[32m{"Navegando para a próxima página"}\u001b[0m')
                sleep(5)
            except:
                self.driver.quit()
        
        print(f'\u001b[32m{"Resultados:"}\u001b[0m')
        print(self.lista_nome_hoteis)
        print(self.lista_preco_hoteis)

        for nome in self.lista_nome_hoteis:
            nome_hoteis_desp.append(nome)
        for preco in self.lista_preco_hoteis:
            preco_hoteis_desp.append(preco)

start = ScrappyDesp()
start.iniciar()

Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Navegando para a próxima página
Resultados:
['Hotel Ryan', 'Hotel Itália', 'Hotel Pires', 'Hotel Marimar The Place', 'Hotel Sibara Flat & Convenções', 'Hotel Plaza Camboriú', 'Marambaia Hotel e Convenções', 'Hotel Palmas Executivo', 'Hotel dos Açores', 'Hotel Miramar', 'Reserva Praia Hotel', 'Atobá Praia Hotel', 'Brut by Slaviero Hotéis', 'Hotel Vieiras', 'Mercure Camboriu Internacional', 

In [15]:
# Lista nome hotéis

print(nome_hoteis_desp)

['Hotel Ryan', 'Hotel Itália', 'Hotel Pires', 'Hotel Marimar The Place', 'Hotel Sibara Flat & Convenções', 'Hotel Plaza Camboriú', 'Marambaia Hotel e Convenções', 'Hotel Palmas Executivo', 'Hotel dos Açores', 'Hotel Miramar', 'Reserva Praia Hotel', 'Atobá Praia Hotel', 'Brut by Slaviero Hotéis', 'Hotel Vieiras', 'Mercure Camboriu Internacional', 'Tropikalya Gold Hotel', 'Hotel Bella Camboriú', 'Slim Balneário Camboriú by Slaviero Hotéis', 'Hamburgo Palace Hotel', 'Santa Inn Hotel', 'Costa Sul Beach Hotel', 'Hotel Melo', 'Hotel Tropikalya Prime', 'Hotel Praiamar', 'Felissimo Exclusive Hotel', 'Hotel Torresol', 'Hotel Blumenhof Balneário Camboriú', 'Hotel Geranium', 'Rezende Suítes Hotel', 'Pousada Ponta do Lobo', 'H Hotel', 'HOTEL SAN REMO INTERNACIONAL', 'Mar Hotel', 'Pousada Villa Atlântica', 'Hotel Shallom', 'Hotel Torre Sol', 'Rieger Balneario Camboriu', 'Apartamento Edifício República Argentina - Balneário Camboriú', 'Hotel Visual', 'Flat Concept', 'Rizzu Marina Hotel', 'Pousada do

In [16]:
# Tratamento formato

nome_hoteis_desp_temp = list(pd.Series(nome_hoteis_desp).str.upper())

nome_hoteis_desp_fmtg = []

for nome in nome_hoteis_desp_temp:
    nome_hoteis_desp_fmtg.append(unidecode(nome))

print(nome_hoteis_desp_fmtg)

['HOTEL RYAN', 'HOTEL ITALIA', 'HOTEL PIRES', 'HOTEL MARIMAR THE PLACE', 'HOTEL SIBARA FLAT & CONVENCOES', 'HOTEL PLAZA CAMBORIU', 'MARAMBAIA HOTEL E CONVENCOES', 'HOTEL PALMAS EXECUTIVO', 'HOTEL DOS ACORES', 'HOTEL MIRAMAR', 'RESERVA PRAIA HOTEL', 'ATOBA PRAIA HOTEL', 'BRUT BY SLAVIERO HOTEIS', 'HOTEL VIEIRAS', 'MERCURE CAMBORIU INTERNACIONAL', 'TROPIKALYA GOLD HOTEL', 'HOTEL BELLA CAMBORIU', 'SLIM BALNEARIO CAMBORIU BY SLAVIERO HOTEIS', 'HAMBURGO PALACE HOTEL', 'SANTA INN HOTEL', 'COSTA SUL BEACH HOTEL', 'HOTEL MELO', 'HOTEL TROPIKALYA PRIME', 'HOTEL PRAIAMAR', 'FELISSIMO EXCLUSIVE HOTEL', 'HOTEL TORRESOL', 'HOTEL BLUMENHOF BALNEARIO CAMBORIU', 'HOTEL GERANIUM', 'REZENDE SUITES HOTEL', 'POUSADA PONTA DO LOBO', 'H HOTEL', 'HOTEL SAN REMO INTERNACIONAL', 'MAR HOTEL', 'POUSADA VILLA ATLANTICA', 'HOTEL SHALLOM', 'HOTEL TORRE SOL', 'RIEGER BALNEARIO CAMBORIU', 'APARTAMENTO EDIFICIO REPUBLICA ARGENTINA - BALNEARIO CAMBORIU', 'HOTEL VISUAL', 'FLAT CONCEPT', 'RIZZU MARINA HOTEL', 'POUSADA DO

In [17]:
# Lista preço hotéis

print(preco_hoteis_desp)

['580', '550', '432', '684', '588', '746', '788', '454', '540', '704', '976', '562', '646', '342', '1.158', '444', '454', '360', '544', '1.270', '582', '530', '420', '348', '2.292', '396', '452', '444', '314', '540', '446', '302', '274', '398', '418', '424', '726', '838', '390', '704', '1.098', '390', '592', '600', '242']


In [18]:
# Tratamento formato

preco_hoteis_desp_nofloat = []
for x in preco_hoteis_desp:
    item = x
    for y in ["."]:
        item = item.replace(y, "")
    preco_hoteis_desp_nofloat.append(item)
print(preco_hoteis_desp_nofloat)

['580', '550', '432', '684', '588', '746', '788', '454', '540', '704', '976', '562', '646', '342', '1158', '444', '454', '360', '544', '1270', '582', '530', '420', '348', '2292', '396', '452', '444', '314', '540', '446', '302', '274', '398', '418', '424', '726', '838', '390', '704', '1098', '390', '592', '600', '242']


In [32]:
# Tratamento formato

preco_hoteis_desp_num = pd.to_numeric(preco_hoteis_desp_nofloat, errors='coerce')
print(preco_hoteis_desp_num)

[ 580  550  432  684  588  746  788  454  540  704  976  562  646  342
 1158  444  454  360  544 1270  582  530  420  348 2292  396  452  444
  314  540  446  302  274  398  418  424  726  838  390  704 1098  390
  592  600  242]


In [33]:
# Mesclagem nomes e preços em dicionário

data_hoteis_desp = []
for el in zip(nome_hoteis_desp_fmtg, preco_hoteis_desp_num):
    data_hoteis_desp.append(el)
data_hoteis_desp_dict = dict(data_hoteis_desp)
print(data_hoteis_desp_dict)

{'HOTEL RYAN': 580, 'HOTEL ITALIA': 550, 'HOTEL PIRES': 432, 'HOTEL MARIMAR THE PLACE': 684, 'HOTEL SIBARA FLAT & CONVENCOES': 588, 'HOTEL PLAZA CAMBORIU': 746, 'MARAMBAIA HOTEL E CONVENCOES': 788, 'HOTEL PALMAS EXECUTIVO': 454, 'HOTEL DOS ACORES': 540, 'HOTEL MIRAMAR': 704, 'RESERVA PRAIA HOTEL': 976, 'ATOBA PRAIA HOTEL': 562, 'BRUT BY SLAVIERO HOTEIS': 646, 'HOTEL VIEIRAS': 342, 'MERCURE CAMBORIU INTERNACIONAL': 1158, 'TROPIKALYA GOLD HOTEL': 444, 'HOTEL BELLA CAMBORIU': 454, 'SLIM BALNEARIO CAMBORIU BY SLAVIERO HOTEIS': 360, 'HAMBURGO PALACE HOTEL': 544, 'SANTA INN HOTEL': 1270, 'COSTA SUL BEACH HOTEL': 582, 'HOTEL MELO': 530, 'HOTEL TROPIKALYA PRIME': 420, 'HOTEL PRAIAMAR': 348, 'FELISSIMO EXCLUSIVE HOTEL': 2292, 'HOTEL TORRESOL': 396, 'HOTEL BLUMENHOF BALNEARIO CAMBORIU': 452, 'HOTEL GERANIUM': 444, 'REZENDE SUITES HOTEL': 314, 'POUSADA PONTA DO LOBO': 540, 'H HOTEL': 446, 'HOTEL SAN REMO INTERNACIONAL': 302, 'MAR HOTEL': 274, 'POUSADA VILLA ATLANTICA': 398, 'HOTEL SHALLOM': 418, 

In [34]:
# Conversão dicionário para dataframe

df_data_hoteis_desp = pd.DataFrame(list(data_hoteis_desp_dict.items()),
                   columns=['Nome', 'Preço Decolar'])
display(df_data_hoteis_desp)

,Nome,Preço Decolar
0,HOTEL RYAN,580
1,HOTEL ITALIA,550
2,HOTEL PIRES,432
3,HOTEL MARIMAR THE PLACE,684
4,HOTEL SIBARA FLAT & CONVENCOES,588
5,HOTEL PLAZA CAMBORIU,746
6,MARAMBAIA HOTEL E CONVENCOES,788
7,HOTEL PALMAS EXECUTIVO,454
8,HOTEL DOS ACORES,540
9,HOTEL MIRAMAR,704


In [35]:
# Merge data

data_merge = pd.merge(df_data_hoteis_bkg, df_data_hoteis_desp, on=['Nome'], how='left')
display(data_merge)

,Nome,Preço Booking,Preço Decolar
0,"LINDO APTO, 100METROS DA PRAIA, NA AV BRASIL C...",396,NaN
1,QUARTO DE CASAL ROMANTICO COM TERRACO,500,NaN
2,MORADA DO ESTALEIRO,609,NaN
3,APTO BC - 500M DA PRAIA,710,NaN
4,3 SUITES LUXUOSO LINDA VISTA MAR A 50M DA PRAIA,1300,NaN
...,...,...,...
223,LINDO APARTAMENTO FRENTE MAR!,3240,NaN
224,CASA DE PRAIA,3300,NaN
225,APTO NA PRAIA CENTRAL EM BALNEARIO CAMBORIU SC,3446,NaN
226,COBERTURA DUPLEX BALNEARIO CAMBORIU,5400,NaN


In [36]:
# Limpeza NaN

data_merge_notna = data_merge.dropna()
display(data_merge_notna)

,Nome,Preço Booking,Preço Decolar
27,RIZZU MARINA HOTEL,1070,1098.0
32,SANTA INN HOTEL,1208,1270.0
65,RESERVA PRAIA HOTEL,972,976.0
79,HOTEL BLUMENHOF BALNEARIO CAMBORIU,378,452.0
83,POUSADA VILLA ATLANTICA,398,398.0
86,TROPIKALYA GOLD HOTEL,440,444.0
87,POUSADA DO BOSQUE DI SICA,448,390.0
88,HOTEL BELLA CAMBORIU,450,454.0
96,POUSADA PONTA DO LOBO,720,540.0
108,HOTEL MARIMAR THE PLACE,682,684.0


In [37]:
# Comparação de preços

df_rate_comparison = data_merge_notna
display(df_rate_comparison)

,Nome,Preço Booking,Preço Decolar
27,RIZZU MARINA HOTEL,1070,1098.0
32,SANTA INN HOTEL,1208,1270.0
65,RESERVA PRAIA HOTEL,972,976.0
79,HOTEL BLUMENHOF BALNEARIO CAMBORIU,378,452.0
83,POUSADA VILLA ATLANTICA,398,398.0
86,TROPIKALYA GOLD HOTEL,440,444.0
87,POUSADA DO BOSQUE DI SICA,448,390.0
88,HOTEL BELLA CAMBORIU,450,454.0
96,POUSADA PONTA DO LOBO,720,540.0
108,HOTEL MARIMAR THE PLACE,682,684.0


In [25]:
df_rate_comparison['% Diferença'] = 0
for el in df_rate_comparison:
    dif = (df_rate_comparison['Preço Decolar'] / df_rate_comparison['Preço Booking'] -1 ) * 100
    df_rate_comparison['% Diferença'] = round(dif, 0)
display(df_rate_comparison)

/tmp/ipykernel_14414/3167378679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rate_comparison['% Diferença'] = 0
/tmp/ipykernel_14414/3167378679.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rate_comparison['% Diferença'] = round(dif, 0)


,Nome,Preço Booking,Preço Decolar,% Diferença
27,RIZZU MARINA HOTEL,1070,1098.0,3.0
32,SANTA INN HOTEL,1208,1270.0,5.0
65,RESERVA PRAIA HOTEL,972,976.0,0.0
79,HOTEL BLUMENHOF BALNEARIO CAMBORIU,378,452.0,20.0
83,POUSADA VILLA ATLANTICA,398,398.0,0.0
86,TROPIKALYA GOLD HOTEL,440,444.0,1.0
87,POUSADA DO BOSQUE DI SICA,448,390.0,-13.0
88,HOTEL BELLA CAMBORIU,450,454.0,1.0
96,POUSADA PONTA DO LOBO,720,540.0,-25.0
108,HOTEL MARIMAR THE PLACE,682,684.0,0.0


In [26]:
# Highlight adversas

def highlight_adversas(s):
    color_red = 'red'
    color_yellow = 'yellow'
    color_green = 'green'
    if s >= 15:
        return 'background-color: %s' % color_red
    elif s < 15 and s > 0:
        return 'background-color: %s' % color_yellow
    elif s <= 0:
        return 'background-color: %s' % color_green

reporte_adversas = df_rate_comparison.style.applymap(highlight_adversas, subset=pd.IndexSlice[:, ['% Diferença']])

In [27]:
# Conversão dataframe para XLSX

writer = pd.ExcelWriter('Reporte Paridade Booking - {}.xlsx'.format(nome_destino))
reporte_adversas.to_excel(writer, f'IN_{checkin} - OUT_{checkout}', index=False)
writer.save()

In [28]:
# % adversas

medicoes = 0
for m in df_rate_comparison['% Diferença']:
    medicoes += 1

adversas = 0
for a in df_rate_comparison['% Diferença']:
    if a > 0:
        adversas += 1

percent_adversas = (adversas / medicoes) * 100

In [29]:
# Calcula o nº de noites

from datetime import datetime

d2 = datetime.strptime(checkout, '%Y-%m-%d')
d1 = datetime.strptime(checkin, '%Y-%m-%d')
n_diarias = abs((d2 - d1).days)

In [30]:
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [31]:
# Dispara e-mail

anexo = "Reporte Paridade Booking - {}.xlsx".format(nome_destino)

def sendMail():
    try:
        subject = "Reporte Paridade Booking - {}".format(nome_destino)
        body = f"Destino: {nome_destino}\n\nCheck-in: {checkin}\nCheck-out: {checkout}\nLOS: {n_diarias}\nPax: {pax}\n\nPercentual adversas: {percent_adversas:.2f}%"
        sender_email = "daniel.farias@decolar.com"
        password = ""
        receiver_email = "daniel.farias@decolar.com"

        message = MIMEMultipart()

        message["From"] = sender_email
        message["To"] = receiver_email
        message["Subject"] = subject

        message.attach(MIMEText(body, "plain"))

        filename = anexo

        with open(filename, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())

        encoders.encode_base64(part)

        part.add_header(
            "Content-Disposition",
            f"attachment; filename = {filename}",
        )

        message.attach(part)
        text = message.as_string()

        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, text)
        
        print('E-mail enviado com sucesso!')
    except:
        print('Erro ao enviar e-mail')

sendMail()

E-mail enviado com sucesso!
